# LetsGrowMoreVIP-Data Science Intern
# Author-Ritesh Meshram
# Next Word Prediction(ADVANCE LEVEL TASK)

In [1]:
#Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
from nltk.tokenize import RegexpTokenizer 
import collections
import pickle
import heapq

In [2]:
#loading the data to the variable
df = '1661-0.txt'
with open(df,encoding='utf-8') as f:
    text= f.readlines()

In [3]:
#Reading and converting data to lower case
text = open(df,encoding='utf-8').read().lower()
print('corpus length:', len(text))


corpus length: 581136


In [5]:
#Tokenizing the data and converting to tokens
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [6]:
#Extracting the unique words using the np.unique function
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [7]:
#Finding the previous and word_next words
LENGTH = 5
word_prev = []
word_next= []
for i in range(len(words) - LENGTH):
    word_prev.append(words[i:i + LENGTH])
    word_next.append(words[i + LENGTH])
#Printing the previous and word_next words
print("Previous :",word_prev[0])
print("word_next :",word_next[0])

Previous : ['contents', 'i', 'a', 'scandal', 'in']
word_next : bohemia


In [8]:
#Creating the array with number of zeros
X = np.zeros((len(word_prev), LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(word_next), len(unique_words)), dtype=bool)
#Properly indexing the previous and next words
for i, each_words in enumerate(word_prev):
    for j, word in enumerate(each_words):
        X[i, j, unique_word_index[word]] = 1
    Y[i, unique_word_index[word_next[i]]] = 1

In [9]:
print(X[0][0])
print(Y[0][0])

[False False False ... False False False]
False


In [10]:
#Selecting the Sequential model 
model = Sequential()
model.add(LSTM(128, input_shape=(LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
#setting Softmax Activation function
model.add(Activation('softmax'))
#setting the RMSprop Optimizer
optimizer = RMSprop(learning_rate=0.01)

In [11]:
#Compiling the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               4260864   
                                                                 
 dense (Dense)               (None, 8193)              1056897   
                                                                 
 activation (Activation)     (None, 8193)              0         
                                                                 
Total params: 5,317,761
Trainable params: 5,317,761
Non-trainable params: 0
_________________________________________________________________


In [13]:
#Training the model.
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

Epoch 1/2
810/810 [==============================] - 243s 295ms/step - loss: 6.0149 - accuracy: 0.1072 - val_loss: 7.1835 - val_accuracy: 0.0988
Epoch 2/2
810/810 [==============================] - 249s 308ms/step - loss: 5.7727 - accuracy: 0.1479 - val_loss: 8.0750 - val_accuracy: 0.1067


In [14]:
#Saving the model as h5 file
model.save('next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
#Loading the model
model = load_model('next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [16]:
#Defining a function to prepare input for using in model
def prep(text):
    x = np.zeros((1, LENGTH, len(unique_words)))
    for a, word in enumerate(text.split()):
        print(word)
        x[0, a, unique_word_index[word]] = 1
    return x

In [17]:
#using the prep funtion we are transforming the data
prep("It is not a lack".lower())

it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [18]:
#Defining a function called sample
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [19]:
#Defining a function predict_completion to predict the data using model
def completion(text):
    original_text = text
    generated = text
    completion = ''
    indices_char={}
    while True:
        x = prep(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion
def completions(text, n=3):
    x = prep(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] + completion(text[1:] + completion[idx]) for idx in next_indices]

In [20]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prep(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

## Next word prediction

In [21]:
q =  "Your life will never be the same again"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))


correct sentence:  Your life will never be the same again
Sequence:  your life will never be
your
life
will
never
be
next possible words:  ['to', 'in', 'so', 'and', 'a']
